In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%cd /content/drive/MyDrive/MyPicture/final_project
!python tkit.py

/content/drive/MyDrive/MyPicture/final_project


In [5]:
#@title Default title text
import tensorflow as tf
import numpy as np
import nltk
tf.compat.v1.enable_eager_execution()
from nltk.stem.lancaster import LancasterStemmer
import json
import random
from tkit import tokenize, stem_lower, encode
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from keras.layers import LeakyReLU
nltk.download('punkt')
from tensorflow.keras.optimizers import SGD
stemmer = LancasterStemmer()

with open('/content/drive/MyDrive/MyPicture/final_project/intents.json') as file_data:
    data = json.load(file_data)


words_list=[]
tags_list=[]
doc_x=[]
doc_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        word=tokenize(pattern)
        # print('\n',word)
        words_list.extend(word)
        doc_x.append(word)
        doc_y.append(intent['tag'])
    
    if (intent['tag'] not in tags_list):
        tags_list.append(intent['tag'])

character_skip=['?',',','.','!']

words_list=[stem_lower(word.lower()) for word in words_list if word not in character_skip]

words_list=sorted(set(words_list)) ### sắp xếp theo Alphabet và bỏ những từ trùng nhau 
tags_list= sorted(tags_list)


x_train=[]  
y_train=[]

data_empty=[0 for i in range (len(tags_list))]

for x, sentence in enumerate(doc_x):
    bag=[] 
    wod = [stem_lower(w) for w in sentence]
    for w in words_list:
        if w in wod:
            bag.append(1)
        else: bag.append(0)
  

    output = data_empty[:]
    output[tags_list.index(doc_y[x])]=1

    x_train.append(bag)     
    y_train.append(output)

x_train= np.array(x_train)
y_train=np.array(y_train)

with open("training_data", "wb") as f:
    pickle.dump({'words':words_list,'topics': tags_list,'x_train': x_train ,'y_train': y_train},f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))
 
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
 
hist = model.fit(x_train, y_train, epochs=200, batch_size=5, verbose=1)
# model.save('chatbot_model.h5', hist)
model.save('/content/drive/MyDrive/MyPicture/final_project/model_train.h5')

Epoch 1/200
22/22 [==============================] - 5s 4ms/step - loss: 3.6253 - accuracy: 0.0189
Epoch 2/200
22/22 [==============================] - 0s 3ms/step - loss: 3.5665 - accuracy: 0.0566
Epoch 3/200
22/22 [==============================] - 0s 3ms/step - loss: 3.5310 - accuracy: 0.0755
Epoch 4/200
22/22 [==============================] - 0s 3ms/step - loss: 3.4251 - accuracy: 0.1415
Epoch 5/200
22/22 [==============================] - 0s 4ms/step - loss: 3.4374 - accuracy: 0.0943
Epoch 6/200
22/22 [==============================] - 0s 3ms/step - loss: 3.3336 - accuracy: 0.1038
Epoch 7/200
22/22 [==============================] - 0s 3ms/step - loss: 3.2755 - accuracy: 0.1415
Epoch 8/200
22/22 [==============================] - 0s 3ms/step - loss: 3.1305 - accuracy: 0.1887
Epoch 9/200
22/22 [==============================] - 0s 3ms/step - loss: 2.9986 - accuracy: 0.1887
Epoch 10/200
22/22 [==============================] - 0s 3ms/step - loss: 2.9233 - accuracy: 0.1792
Epoch 11/